So the goal of this project was to pull Tweets for popular brands with large followings on social and analyze those tweets to understand the relationship between tweet "factors" (length, number of hashtags, @ mentions, etc.) and tweet performace (as defined as retweets). The below code is for pulling the Tweets from the Twitter API via a package called Tweepy and then attempting to pull out the important metadata parts. 
The last part is happening with mixed success. Right now I'm having trouble pulling the "media" data from the Tweet entity data.
https://dev.twitter.com/overview/api/entities#obj-url explains the tweet entity and metadata within it


Cells 20,21,36 work to call the Twitter API and pull 3200 tweets from each brand listed in handle_list
Cells 41 through 47 are calling back in the CSV 36 exported and messing with it
Cell 48 outlines what needs to come

The two cells below 48 are attempts to include the media metadata I'm having trouble calling. It's only pulling from the CocaCola brand so as to speed processing time and not piss off the Twitter API

In [20]:
import tweepy  #tweepy is a package that lets you interact with the Twitter API
import csv
import string
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
import time
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
#Twitter API credentials
consumer_key = "IocNJcYUVIJJ66wSaIXF5CXDu"
consumer_secret = "AEi4U7qCFk9hnlOwweRjFu8f8dLK12D7iX8lQ0s0BkIxnYMWQV"
access_key = "289116268-4jPFCQ6BiWrFAHY4CxIff9NDLQTwrqCodti8V4ad"
access_secret = "rLUz0MhPkzl20XENm4Di7cjtLeE0FBpVQENQ9pO7BiGHW"

In [36]:
##works

handles_list = ["CocaCola", "Nike", "MonsterEnergy", "RedBull", "GoPro", "Oreo", "NatGeo", "Audi", "MercedesBenz"]

def get_all_tweets(screen_name, writer): 
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    alltweets = []
    
    new_tweets = api.user_timeline(screen_name = handle,count=200)
    
    alltweets.extend(new_tweets)
    
    oldest = alltweets[-1].id -1
    
    while len(new_tweets) > 0:
        print "getting tweets before %s" %(oldest)
        
        new_tweets = api.user_timeline(screen_name = handle,count=200,max_id=oldest)
        
        alltweets.extend(new_tweets)
        
        oldest = alltweets[-1].id -1
        
        print "...%s tweets downloaded so far" % (len(alltweets))
    outtweets = [[tweet.id_str, tweet.created_at, tweet.user.screen_name, tweet.user.id, tweet.text.encode("utf-8"), tweet.retweet_count, tweet.favorite_count, tweet.in_reply_to_status_id, len(tweet.entities['hashtags']), len(tweet.entities['user_mentions'])] for tweet in alltweets]
    writer.writerow(["tweet id","created_at","user name", "user_id","text","retweet count","favorite count","in reply to status", "tweet hashtag count", "tweet mentions count"])
    writer.writerows(outtweets)

    pass



if __name__ == '__main__':
    with open('all_tweets.csv', 'w') as f_all:
        writer = csv.writer(f_all)

        for handle in handles_list:
            get_all_tweets("handles", writer)
            print ("Done.")
        

getting tweets before 652849171351912447
...400 tweets downloaded so far
getting tweets before 647304412449402879
...600 tweets downloaded so far
getting tweets before 643108663054495743
...800 tweets downloaded so far
getting tweets before 638425704611319808
...1000 tweets downloaded so far
getting tweets before 637054297008644095
...1200 tweets downloaded so far
getting tweets before 635897804376920064
...1400 tweets downloaded so far
getting tweets before 634824772698947584
...1600 tweets downloaded so far
getting tweets before 633845125937000447
...1800 tweets downloaded so far
getting tweets before 632295883145650175
...2000 tweets downloaded so far
getting tweets before 631209703297695743
...2200 tweets downloaded so far
getting tweets before 629016116363403263
...2400 tweets downloaded so far
getting tweets before 627180672566853631
...2600 tweets downloaded so far
getting tweets before 625723912987480063
...2800 tweets downloaded so far
getting tweets before 624037634306912255


In [ ]:
#need to exclude @mentions from tweet collections
#need to break out tweet entities

In [41]:
df = pd.read_csv("all_tweets.csv")

In [42]:
df.head()

,tweet id,created_at,user name,user_id,text,retweet count,favorite count,in reply to status,tweet hashtag count,tweet mentions count
0,661722004450623488,2015-11-04 01:49:54,CocaCola,26787673,@trey_a_miller Pls send us a detailed message...,0,0,661676108346191872,0,1
1,661710094426636288,2015-11-04 01:02:35,CocaCola,26787673,@dbquark Pls send us a message via https://t....,0,0,661675722021412864,0,1
2,661610045780660224,2015-11-03 18:25:01,CocaCola,26787673,"@RAPIRAMOS Rapi, nos encanta que asi alla sido...",3,1,661596424904151040,0,1
3,661550141531414528,2015-11-03 14:26:59,CocaCola,26787673,"@evetrakway Eve, you can send us a message at:...",2,1,661512888729927680,0,1
4,661498502816624640,2015-11-03 11:01:47,CocaCola,26787673,"@FranEbner Pancho, ponte en contacto con nuest...",1,1,660624225502392321,0,1


In [43]:
df.shape

(29419, 10)

In [44]:
#how many @replies are in the file. If they outnumber the public facing tweets should edit to remove 
#since public facing tweets are the ones that will get higher retweet/favorite metrics
df['in reply to status'].isnull().sum()

16129

In [47]:
#create dataframe of only public facing tweets
df_public_facing = df[(df['in reply to status'].isnull())]
df_public_facing.head()

,tweet id,created_at,user name,user_id,text,retweet count,favorite count,in reply to status,tweet hashtag count,tweet mentions count
15,661057012592078848,2015-11-02 05:47:28,CocaCola,26787673,"Take a sip, flip the bottle, walk it off. #Con...",161,375,NaN,2,0
24,660887343247683584,2015-11-01 18:33:15,CocaCola,26787673,Today you have 60 more minutes to do whatever ...,483,1205,NaN,1,0
31,660463803738386432,2015-10-31 14:30:16,CocaCola,26787673,"Careful, they sometimes run in packs. #WereCok...",188,323,NaN,2,0
48,659762836894126080,2015-10-29 16:04:52,CocaCola,26787673,I iz delishuz? #BottleCapYourCat #NationalCatD...,1379,2951,NaN,2,0
59,659157764522962944,2015-10-28 00:00:32,CocaCola,26787673,Noticing a huge spike in people who want to be...,558,1544,NaN,0,0


In [48]:
df_public_facing.shape

(16129, 10)

For analysis end goal is to predict retweets (or favorites, but starting with retweets) 
So need to break out tweet posting date (day of week, time of day, month), length of tweet, hashtag count, @mention count
Overall and by brand

*Obviously don't have paid so analysis will be skewed. Maybe look at distribution 

To do:
Time breakout
Need URL count


In [ ]:
#.append attempt
#other iteration below worked aside from inability to grab 'media' objects

def get_all_tweets(screen_name):
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    alltweets = []
    
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    alltweets.extend(new_tweets)
    
    oldest = alltweets[-1].id -1
    
    while len(new_tweets) > 0:
        print "getting tweets before %s" %(oldest)
        
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        alltweets.extend(new_tweets)
        
        oldest = alltweets[-1].id -1
        
        print "...%s tweets downloaded so far" % (len(alltweets))
      
    hashtag_count = []
    hashtag = []
    for tweet in alltweets:
        tweet_id = tweet.id
        content = tweet.text
        content = content.replace('\n', '')
        created_at = tweet.created_at
        user_name = tweet.user.screen_name
        user_id = tweet.user.id
        user_followers = tweet.user.followers_count
        retweet_count = tweet.retweet_count
        favorite_count = tweet.favorite_count
        in_reply_to_screen_name = tweet.in_reply_to_screen_name
        in_reply_to_status_id = tweet.in_reply_to_status_id
        
        tweet_urls, tweet_expanded_urls, tweet_hashtags, tweet_mentions, media_expanded_url = [], [], [], [], []

        if 'hashtag' in tweet.entities:
            tweet_hashtag_count = len(tweet.entities['hashtags'])
            hashtag_count.append(len(tweet.entities['hashtags']))
        else:
            tweet_hashtag_count = ''
            hashtag_count.append(0)
        if 'hashtag' in tweet.entities:
            tweet_hashtag = tweet.entities['hashtags']['text']
            hashtag.append(tweet.entities['hashtags']['text'])
        else:
            tweet_hashtag = ''
        if 'screen_name' in tweet.entities['user_mentions']:
            tweet_mention_count = len(tweet.entities['user_mentions'])
        else:
            tweet_mention_count = ''
        if 'symbols' in tweet.entities:
            tweet_symbols = tweet.entities['symbols']
        else:
            tweet_symbols = ''
            
        if 'media' in tweet.entities:
            tweet_media_count = len(tweet.entities['media'])
        else:
            tweet_media_count = ''
        if 'media' in tweet.entities:
            if 'expanded_url' in tweet.entities['media'][0]:
                media_expanded_url = tweet.entities['media'][0]['expanded_url']
            else:
                media_expanded_url = ''
            if 'media_url' in tweet.entities['media'][0]:
                media_url = tweet.entities['media'][0]['media_url']
            else:
                media_url = ''
            if 'type' in tweet.entities['media'][0]:
                media_type = tweet.entities['media'][0]['type']
            else:
                media_type = ''
        else:
            media_type = ''
            media_url = ''
            media_expanded_url = ''
            
        if 'url' in tweet.entities:
            tweet_url_count = len(tweet.entities['urls'])
        else:
            tweet_url_count = ''


    outtweets = [[tweet.id_str, tweet.created_at, tweet.user.screen_name, tweet.user.id, tweet.text.encode("utf-8"), tweet.retweet_count, tweet.favorite_count, tweet.in_reply_to_status_id] for tweet in alltweets]
    outtweets['hashtag_count'] = np.array(hashtag_count)
    with open('%s_tweets.csv' % screen_name, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["tweet id","created_at","user name", "user_id","text","retweet count","favorite count","in reply to status", "tweet entities"])
        writer.writerows(outtweets)
        
    pass

if __name__ == '__main__':
        #grab tweets from this username
        get_all_tweets("CocaCola")

In [19]:
#going to try to for loop the outtweets by using tweet.entities[hashtag][count] or whatever the code is
#worked and implemented in the all tweets csv creation function

def get_all_tweets(screen_name): 
    #authorize twitter to get tweets, start tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #list to hold tweets to be pulled
    alltweets = []
    
    #make initial request for tweets. 
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
            
    
    #save tweets getting pulled to our list
    alltweets.extend(new_tweets)
    
    #value for oldest tweet minus one
    oldest = alltweets[-1].id -1
    
    #reiterate the above grabbing tweet one older than the last 200 grabbed
    while len(new_tweets) > 0:
        print "getting tweets before %s" %(oldest)
        
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        alltweets.extend(new_tweets)
        
        oldest = alltweets[-1].id -1
        
        print "...%s tweets downloaded so far" % (len(alltweets))
    
    
    tweet_url_include = []
    media_type = []
    for tweet in alltweets:
        if media in tweet.entities:
            if 'type' in tweet.entities['media'][0]:
                media_format = tweet.entities['media'][0]['type']
                media_type.append(media_format)
            else:
                media_format = ''
        else:
            media_type = ''
        if url in tweet.entities:
            if 'url' in tweet.entities['url']:
                include_url = tweet.entities['url']['url']
                tweet_url_include.append(include_url)
            else:
                include_url = ''
        
    #put all of the above in an array
    outtweets = [[tweet.id_str, tweet.created_at, tweet.user.screen_name, tweet.user.id, tweet.text.encode("utf-8"), tweet.retweet_count, tweet.favorite_count, tweet.in_reply_to_status_id, tweet.retweeted, len(tweet.entities['hashtags']), len(tweet.entities['user_mentions'])] for tweet in alltweets]
    outtweets['url_in_tweet'] = np.array(tweet_url_include)
    outtweets['tweet_media_type'] = np.array(media_type)
    #write csv from the array
    with open('%s_tweets.csv' % screen_name, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["tweet id","created_at","user name", "user_id","text","retweet count","favorite count","in reply to status", "is retweet", "tweet hashtag count", "tweet mentions count", "tweet url count", "tweet media type", "url in tweet", "media type"])
        writer.writerows(outtweets)

    pass


if __name__ == '__main__':
        #grab tweets from this username
        get_all_tweets("CocaCola")

getting tweets before 653252461164130303
...400 tweets downloaded so far
getting tweets before 647333771109384191
...600 tweets downloaded so far
getting tweets before 643200511366275073
...800 tweets downloaded so far
getting tweets before 638427811670310911
...1000 tweets downloaded so far
getting tweets before 637078012748562431
...1200 tweets downloaded so far
getting tweets before 635904120218632191
...1400 tweets downloaded so far
getting tweets before 634826078528405503
...1600 tweets downloaded so far
getting tweets before 634098306839355391
...1800 tweets downloaded so far
getting tweets before 632348735415144447
...2000 tweets downloaded so far
getting tweets before 631211701283725311
...2200 tweets downloaded so far
getting tweets before 629018535906422784
...2400 tweets downloaded so far
getting tweets before 627182543553499135
...2600 tweets downloaded so far
getting tweets before 625729786950877184
...2800 tweets downloaded so far
getting tweets before 624043535994626048


NameError: global name 'media' is not defined

In [26]:
cc = pd.read_csv("CocaCola_tweets.csv")
cc.head()

,tweet id,created_at,user name,user_id,text,retweet count,favorite count,in reply to status,tweet hashtag count,tweet mentions count
0,661722004450623488,2015-11-04 01:49:54,CocaCola,26787673,@trey_a_miller Pls send us a detailed message...,0,0,6.616761e+17,0,1
1,661710094426636288,2015-11-04 01:02:35,CocaCola,26787673,@dbquark Pls send us a message via https://t....,0,0,6.616757e+17,0,1
2,661610045780660224,2015-11-03 18:25:01,CocaCola,26787673,"@RAPIRAMOS Rapi, nos encanta que asi alla sido...",3,1,6.615964e+17,0,1
3,661550141531414528,2015-11-03 14:26:59,CocaCola,26787673,"@evetrakway Eve, you can send us a message at:...",2,1,6.615129e+17,0,1
4,661498502816624640,2015-11-03 11:01:47,CocaCola,26787673,"@FranEbner Pancho, ponte en contacto con nuest...",1,1,6.606242e+17,0,1
